# Primal Dual

# Projection onto the simplex



<div class="alert alert-block alert-success">
This notebook implements an algorithm for the projection of a vector $y$ onto the simplex $\Delta$ with
\begin{align}
    \Delta := \Big\{ (x_1,\ldots,x_N)\in \mathbb{R}^N:\, \sum_{n=1}^N x_n =a \quad\text{and}\quad x\geq 0 \Big\}.
\end{align}
    
The implemented algorithm was proposed in: Condat, L. <i>Fast projection onto the simplex and the ℓ<sub>1</sub> ball</i>. Math. Program. 158, 575–585 (2016). [https://doi.org/10.1007/s10107-015-0946-6](https://doi.org/10.1007/s10107-015-0946-6). 
    
The paper is also available at [https://hal.archives-ouvertes.fr/hal-01056171v2](https://hal.archives-ouvertes.fr/hal-01056171v2).
</div>

<div style="background-color:rgba(0, 0, 0, 0.0470588); vertical-align: middle; padding:5px 0; padding-left: 40px;">
<h2 style="color: #5e9ca0;">Condat's Algorithm</h2>
<ol>
<li>Set $v:= (y_1)$, $u$ as an empty list, $\rho:= y_1 - a$.</li>
<li>For $n \in \{2,\ldots, N\}$, do
<ol>
<li>If $y_n > \rho$
<ol>
<li>Set $\rho := \rho + (y_n - \rho)/(|v|+1)$.</li>
<li>If $\rho > y_n - a$, add $y_n$ to $v$.</li>
<li>Else, add $v$ to $u$, set $v = (y_n)$, $\rho = y_n -a$.</li>
</ol>
</li>
</ol>
</li>
<li>If $u$ is not empty, for every element $y$ of $u$, do
<ol>
<li>If $y > \rho$, add $y$ to $v$ and set $\rho := \rho + (y-\rho)/|v|$.</li>
</ol>
</li>
<li>Do, while $|v|$ changes,
<ol>
<li>For every element $y$ of $v$ do
<ol>
<li>If $y\leq \rho$, remove $y$ from $v$ and set $\rho := \rho + (\rho - y)/|v|$.</li>
</ol>
</li>
</ol>
</li>
<li>Set $\tau := \rho$, $K = |v|$.</li>
<li>For $n \in \{1,\ldots,N\}$, set $x_n := \max \{y_n - \tau, 0\}$.</li>
</ol>
</div>

The algorithm will be tested projecting a random vector $y$ of size $N$ where $y_n \sim \mathcal{U}(-1,2)$ for all $n\in \{1,\ldots,N\}$.

In [1]:
import numpy as np
from numpy import nditer, ndenumerate, zeros, zeros_like, clip, allclose, full, maximum
from numpy.random import default_rng as rng

In [2]:
N = 1000
y = rng(0).uniform(-1,2,N)
a = 1.0

## List version

<div class="alert alert-block alert-info">
First implementation is list-based. It sticks to the algorithm above, and thus it cannot take the advantage that $v$ is the list of positive values of $y$, thus requiring the unnecessary triming in Step 6.
</div>

In [3]:
# Step 1
v = [y[0]]
u = []
ρ = y[0] - a

In [4]:
# Step 2
for n in range(1,N):
    yₙ = y[n]
    if yₙ > ρ:
        ρ += (yₙ - ρ)/( len(v) + 1 )
        if ρ > yₙ - a:
            v.append(yₙ)
        else:
            u.extend(v)
            v = [yₙ]
            ρ = yₙ - a
# Step 3
if len(u) > 0:
    for ι in iter(u):
        if ι > ρ:
            v.append(ι)
            ρ += (ι - ρ)/( len(v) )
# Step 4
while True:
    ℓ_v = len(v)
    for ı in range(ℓ_v - 1, -1, -1):
        if v[ı] <= ρ:
            ρ += (ρ - v[ı])/(len(v) - 1)
            del v[ı]
    if len(v) >= ℓ_v:
        break

In [5]:
x = clip(y - ρ, 0, None)

In [6]:
v - ρ

array([0.06744738, 0.01832359, 0.06110709, 0.01940269, 0.00520454,
       0.06056962, 0.02164565, 0.01061472, 0.04190357, 0.05789789,
       0.02130254, 0.0148837 , 0.06684845, 0.05188726, 0.00241651,
       0.04805173, 0.0173415 , 0.02582169, 0.03312648, 0.07432163,
       0.02410941, 0.01056192, 0.03883018, 0.03691761, 0.05087173,
       0.00455268, 0.06171698, 0.05232123])

## Vectorial version 1

<div class="alert alert-block alert-info">
The following adaptation introduces some array operations to avoid the extra step of clipping negative numbers. Here $v$ is not a set but a boolean array of the same size as $y$. Its cells determine if a cell of $y$ is active in $v$ or not. In the end, $x$ is computed just on the active cells given by $v$.
</div>

In [7]:
# Step 1
ρ = y[0] - a

v = zeros(N, dtype=bool)
x = zeros_like(y)
u = []
v[0] = True

In [8]:
# Step 2
for n in range(1,N):
    yₙ = y[n]
    if yₙ > ρ:
        ρ += (yₙ - ρ)/( v.sum() + 1 )
        if ρ > yₙ - a:
            v[n] = True
        else:
            u.extend([i[0] for i,j in ndenumerate(v) if j])
            v *= False;    v[n] = True
            ρ = yₙ - a
# Step 3
if len(u) > 0:
    for n in iter(u):
        yₙ = y[n]
        if yₙ > ρ:
            v[n] = True
            ρ += (yₙ - ρ)/( v.sum() )
# Step 4
while True:
    ℓ_v = v.sum()
    for i,j in ndenumerate(v):
        if j:
            if y[i[0]] <= ρ:
                ρ += (ρ - y[i[0]])/(v.sum() - 1)
                v[i[0]] = False
    if v.sum() >= ℓ_v:
        break

In [9]:
x[v] = y[v] - ρ #clip(y[v] - ρ, 0.0, None)

In [10]:
x[v]

array([0.06744738, 0.01832359, 0.06110709, 0.01940269, 0.00520454,
       0.06056962, 0.02164565, 0.01061472, 0.04190357, 0.05789789,
       0.02130254, 0.0148837 , 0.06684845, 0.05188726, 0.00241651,
       0.04805173, 0.0173415 , 0.02582169, 0.03312648, 0.07432163,
       0.02410941, 0.01056192, 0.03883018, 0.03691761, 0.05087173,
       0.00455268, 0.06171698, 0.05232123])

## Vectorial version 2

<div class="alert alert-block alert-info">
The final implementation takes back the original idea of using lists instead of an array of the same size as $y$. In this case, $v$ does not contain values from $y$ but its indeces. This way a sparse representation of $v$ is achieved and $x$ does not go through a clipping step as well.
</div>

In [11]:
# Step 1
ρ = y[0] - a

x = zeros_like(y)
v = [0]
u = []

In [12]:
# Step 2
for n in range(1,N):
    yₙ = y[n]
    if yₙ > ρ:
        ρ += (yₙ - ρ)/( len(v) + 1 )
        if ρ > yₙ - a:
            v.append(n)
        else:
            u.extend(v)
            v = [n]
            ρ = yₙ - a
# Step 3
if len(u) > 0:
    for n in iter(u):
        yₙ = y[n]
        if yₙ > ρ:
            v.append(n)
            ρ += (yₙ - ρ)/( len(v) )
# Step 4
while True:
    ℓ_v = len(v)
    for i,j in enumerate(v):
        if y[j] <= ρ:
            ρ += (ρ - y[j])/(len(v) - 1)
            del v[i]
    if len(v) >= ℓ_v:
        break

In [13]:
x[v] = y[v] - ρ #clip(y[v] - ρ, 0.0, None)

In [14]:
x[v]

array([0.06744738, 0.01832359, 0.06110709, 0.01940269, 0.00520454,
       0.06056962, 0.02164565, 0.01061472, 0.04190357, 0.05789789,
       0.02130254, 0.0148837 , 0.06684845, 0.05188726, 0.00241651,
       0.04805173, 0.0173415 , 0.02582169, 0.03312648, 0.07432163,
       0.02410941, 0.01056192, 0.03883018, 0.03691761, 0.05087173,
       0.00455268, 0.06171698, 0.05232123])

---

## Performance tests

<div class="alert alert-block alert-warning">
Now we check which routine is the fastest.
</div>

In [15]:
def CondatP_a(y,a,N):
    # Step 1
    v = [y[0]]
    u = []
    ρ = y[0] - a
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(yₙ)
            else:
                u.extend(v)
                v = [yₙ]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for ι in iter(u):
            if ι > ρ:
                v.append(ι)
                ρ += (ι - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for ı in range(ℓ_v - 1, -1, -1):
            if v[ı] <= ρ:
                ρ += (ρ - v[ı])/(len(v) - 1)
                del v[ı]
        if len(v) >= ℓ_v:
            break
            
    return clip(y - ρ, 0, None)

def CondatP_b(y,a,N):
    # Step 1
    ρ = y[0] - a

    v = zeros(N, dtype=bool)
    x = zeros_like(y)
    u = []
    v[0] = True
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( v.sum() + 1 )
            if ρ > yₙ - a:
                v[n] = True
            else:
                u.extend([i[0] for i,j in ndenumerate(v) if j])
                v *= False;    v[n] = True
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v[n] = True
                ρ += (yₙ - ρ)/( v.sum() )
    # Step 4
    while True:
        ℓ_v = v.sum()
        for i,j in ndenumerate(v):
            if j:
                if y[i[0]] <= ρ:
                    ρ += (ρ - y[i[0]])/(v.sum() - 1)
                    v[i[0]] = False
        if v.sum() >= ℓ_v:
            break

    x[v] = y[v] - ρ 
    return x

def CondatP_c(y,a,N):
    # Step 1
    ρ = y[0] - a

    x = zeros_like(y)
    v = [0]
    u = []
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(n)
            else:
                u.extend(v)
                v = [n]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v.append(n)
                ρ += (yₙ - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for i,j in enumerate(v):
            if y[j] <= ρ:
                ρ += (ρ - y[j])/(len(v) - 1)
                del v[i]
        if len(v) >= ℓ_v:
            break
    
    x[v] = y[v] - ρ
    return x

In [16]:
%timeit -r 10 -n 200 CondatP_a(y,a,N)

439 µs ± 59.3 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [17]:
%timeit -r 10 -n 200 CondatP_b(y,a,N)

2.57 ms ± 83.8 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [18]:
%timeit -r 10 -n 200 CondatP_c(y,a,N)

505 µs ± 36.8 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


As we can see, the last implementation might be a little faster than the original.

---

## Just-in-time compilation with numba

<div class="alert alert-block alert-warning">
We use numba to speed up the computation time.
</div>

In [19]:
from numba import jit, prange

In [20]:
@jit(nopython=True, fastmath = True, parallel = True)
def CondatP_a(y,a,N):
    # Step 1
    v = [y[0]]
    u = []
    ρ = y[0] - a
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(yₙ)
            else:
                u.extend(v)
                v = [yₙ]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for ι in iter(u):
            if ι > ρ:
                v.append(ι)
                ρ += (ι - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for ı in range(ℓ_v - 1, -1, -1):
            if v[ı] <= ρ:
                ρ += (ρ - v[ı])/(len(v) - 1)
                del v[ı]
        if len(v) >= ℓ_v:
            break
            
    return maximum(y - ρ, 0)

In [21]:
@jit(nopython=True, fastmath = True, parallel = True)
def CondatP_b(y,a,N):
    # Step 1
    ρ = y[0] - a

    v = full(N, False)
    x = zeros_like(y)
    u = []
    v[0] = True
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( v.sum() + 1 )
            if ρ > yₙ - a:
                v[n] = True
            else:
                u.extend([i[0] for i,j in ndenumerate(v) if j])
                v *= False;    v[n] = True
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v[n] = True
                ρ += (yₙ - ρ)/( v.sum() )
    # Step 4
    while True:
        ℓ_v = v.sum()
        for i,j in ndenumerate(v):
            if j:
                if y[i[0]] <= ρ:
                    ρ += (ρ - y[i[0]])/(v.sum() - 1)
                    v[i[0]] = False
        if v.sum() >= ℓ_v:
            break

    x[v] = y[v] - ρ 
    return x

In [22]:
@jit(nopython=True, fastmath = True)#, parallel = True)
def CondatP_c(y,a,N):
    # Step 1
    ρ = y[0] - a

    x = zeros_like(y)
    v = [0]
    u = []
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(n)
            else:
                u.extend(v)
                v = [n]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v.append(n)
                ρ += (yₙ - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for i,j in enumerate(v):
            if y[j] <= ρ:
                ρ += (ρ - y[j])/(len(v) - 1)
                del v[i]
        if len(v) >= ℓ_v:
            break
    
    #x[v] = y[v] - ρ
    for n in iter(v):
        x[n] = y[n] - ρ
    #for n in prange(len(v)):
    #    x[v[n]] = y[v[n]] - ρ
    return x

In [23]:
# Run things once for pre-compiling:
CondatP_a(y,a,N), CondatP_b(y,a,N), CondatP_c(y,a,N);

Now we see a clear winner.

In [24]:
%timeit -r 10 -n 200 CondatP_a(y,a,N)

The slowest run took 4.31 times longer than the fastest. This could mean that an intermediate result is being cached.
114 µs ± 61.4 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [25]:
%timeit -r 10 -n 200 CondatP_b(y,a,N)

KeyboardInterrupt: 

In [ ]:
%timeit -r 10 -n 200 CondatP_c(y,a,N)

Finally, a sanity check:

In [ ]:
allclose(CondatP_a(y,a,N), CondatP_b(y,a,N)), allclose(CondatP_a(y,a,N), CondatP_c(y,a,N))

---

Now we can finally addapt the code to accept a matrix $\gamma$ as input.

In [26]:
N = 1000
M = 500
γ = rng(0).uniform(-1,2,(N,M))
a = 1.0

In [27]:
y = rng(0).uniform(-1,2,N)
x = zeros_like(γ)

In [28]:
@jit(nopython=True, fastmath = True)#, parallel = True)
def CondatP_c(y,x,a,N):
    # Step 1
    ρ = y[0] - a
    v = [0]
    u = []
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(n)
            else:
                u.extend(v)
                v = [n]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v.append(n)
                ρ += (yₙ - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for i,j in enumerate(v):
            if y[j] <= ρ:
                ρ += (ρ - y[j])/(len(v) - 1)
                del v[i]
        if len(v) >= ℓ_v:
            break
    
    #x[v] = y[v] - ρ
    for n in iter(v):
        x[n] = y[n] - ρ
    #for n in prange(len(v)):
    #    x[v[n]] = y[v[n]] - ρ
    return x

In [29]:
CondatP_c(y,x,a,N); # Null test

In [30]:
for i in range(N):
    x[i] = CondatP_c(γ[i],x[i],a,M)

In [31]:
# Just checking :)
for i in range(N):
    if x[i].sum() > 2:
        print(i)

26
27
77
94
163
171
194
198
239
252
275
278
308
349
368
405
412
414
497
530
698
734
736
745
762
790
799
812


In [32]:
x.sum(axis=1)

array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        , 33.17240125,  9.86861806,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.  

# Primal Dual

In [41]:
@jit(nopython=True, fastmath = True)#, parallel = True)
def primal_dual(c,m,n,iters):
    #Algorithm for calculating solution x, in the primal space
    #and y_1, y_2 in the dual space.
    #Also returns the value of the objective function c*x at each
    #iteration.
    
    #Initialise tau
    tau = 0.0001
    
    #Initialise sigma
    sig = 1/(tau*2) - 0.5
    
    #fetch lengths of m and n.
    N = n.size
    M = m.size
    
    #Initialise x
    x = np.zeros((M,N))
    x[:,0] = m
    x[0,:] = n
    x[0,0] = 0.5*(m[-1] + n[-1])
    
    #Initialise y_1 and y_2
    y_1 = zeros((M,N))
    y_2 = zeros((M,N))
    
    y_1[0,:] = n
    y_2[:,0] = m
    
    #Save that objective function
    obj = [np.sum(np.multiply(c,x))]
    
    
    for k in range(iters):
        #update x
        xnew = x - tau*c - tau*(y_1 + y_2)
        
        #matrices for projection over the simplex C_1^m and the simplex C_2^n.
        γ_1 = (1/sig)*y_1 + 2*xnew - x
        γ_2 = (1/sig)*y_2 + 2*xnew - x
        
        
        x_1 = zeros_like(γ_1)
        for i in range(M):
            x_1[i] = CondatP_c(γ_1[i],x_1[i],m[i],N)
        
        x_2 = zeros_like(γ_2)
        for i in range(N):
            x_2[:,i] = CondatP_c(γ_2[:,i],x_2[:,i],n[i],M)
            
        
        #update x using the projection over the simplex C_1^m
        y_1new = y_1 + sig*(2*xnew - x) - sig*x_1
  
        #update x using the projection over the simplex C_2^n
        y_2new = y_2 + sig*(2*xnew - x) - sig*x_2
        
        #Reset x,y_1, y_2 for the next iteration
        x = xnew
        y_1 = y_1new
        y_2 = y_2new
        
        #update objective function
        obj.append(np.sum(np.multiply(c,x)))
        
    return x,obj

In [44]:
import matplotlib.pyplot as plt
import time
x = np.linspace(0,1,1000)
y = np.linspace(1,2,1000)
len_x = np.size(x)
len_y = np.size(y)

c = np.zeros((len_x,len_y))
for i in range(len_x):
    for j in range(len_y):
        c[i,j] = (y[j]-x[i])**2

m = np.ones(len_x)/len_x
n = np.ones(len_y)/len_y
M = len_x
N = len_y

t = time.time()
[x,obj] = primal_dual(c,m,n,500)
elapsed = time.time() - t
print('Time taken for Primal Dual =', elapsed)
print(obj[-1])
#plt.plot(obj)
#plt.imshow(x)

Time taken for Primal Dual = 199.05631470680237
0.9990457115760565


## Exact Test

In [ ]:
import matplotlib.pyplot as plt
#set parameters accordingly.
M = 4                  
N = 4
m = 0.25*np.ones(M)
n = 0.25*np.ones(N)
c = np.array([[3.10112791, 2.01924761, 2.        , 2.01924761],
     [3.10112791, 3.        , 1.16637984, 1.        ],
     [3.        , 3.10112791, 1.16637984, 2.01924761],
     [1.        , 3.        , 2.33275967, 4.22075809]])
[x,obj] = primal_dual(c,m,n,100)
plt.plot(obj)
print(obj[-1])
print(x)

## Generalised Forward-Backward Splitting

<div class="alert alert-block alert-success">
This implements a generalised forward-backward split approach for the OT problem.
    
The implemented algorithm was proposed in: Raguet. H, Fadili. J, Peyre. G <i>Generalised Forward-Backward Splitting</i>. SIAM Journal on Imaging Sciences. (2013)
    
A preprint of the paper is available at https://arxiv.org/pdf/1108.4404.pdf
</div>

In [34]:
@jit(nopython=True, fastmath = True)#, parallel = True)
def generalised_forward_backward(c,m,n,iters):
    #Algorithm for calculating solution x, in the primal space
    #and y_1, y_2 in the dual space.
    #Also returns the value of the objective function c*x at each
    #iteration.
    
    #First compute ?
    mu = np.linalg.norm(c,2)     # 1 -> 10^-1 -> 10^-2 -> ...
    # ? is selected as the midpoint of the interval
    #e = 1/mu #0.5 * 1/mu;        # remove
    # ? does not depend on the current iteration
    gam = 0.0001
    #likewise, we do not require a change in ?
    lam = 1.0;   #1.0
    
    #fetch lengths of m and n.
    N = n.size
    M = m.size
    
    #Initialise x
    #x = np.zeros((M,N))
    #x[:,0] = m
    #x[0,:] = n
    #x[0,0] = 0.5*(m[-1] + n[-1])
    
    #Initialise y_1 and y_2
    v_1 = zeros((M,N))
    v_2 = zeros((M,N))
    
    v_1[0,:] = n
    v_2[:,0] = m
    
    #Initialise x
    x = 0.5*(v_1 + v_2)
    
    #Save that objective function
    obj = [np.sum(np.multiply(c,x))]
    
   
    
    for k in range(iters):
    
        #matrices for projection over the simplex C_1^m and the simplex C_2^n.
        γ_1 = 2*x - v_1 - gam*c
        γ_2 = 2*x - v_2 - gam*c
        
        
        x_1 = zeros_like(γ_1)
        for i in range(M):
            x_1[i] = CondatP_c(γ_1[i],x_1[i],m[i],N)
        
        x_2 = zeros_like(γ_2)
        for i in range(N):
            x_2[:,i] = CondatP_c(γ_2[:,i],x_2[:,i],n[i],M)
            
        
        #update x using the projection over the simplex C_1^m
        v_1new = v_1 + lam*(x_1 - x)
  
        #update x using the projection over the simplex C_2^n
        v_2new = v_2  + lam*(x_2 - x)
        
        #Update x,v_1, v_2 for the next iteration
        v_1 = v_1new
        v_2 = v_2new
        x = 0.5*(v_1 + v_2)
        
        #update objective function
        obj.append(np.sum(np.multiply(c,x)))
        
        
    return x,obj

In [45]:
import matplotlib.pyplot as plt
x = np.linspace(0,1,1000)
y = np.linspace(1,2,1000)
len_x = np.size(x)
len_y = np.size(y)

c = np.zeros((len_x,len_y))
for i in range(len_x):
    for j in range(len_y):
        c[i,j] = (y[j]-x[i])**2

m = np.ones(len_x)/len_x
n = np.ones(len_y)/len_y
N = len_y


t = time.time()
[x,obj] = generalised_forward_backward(c,m,n,500)
elapsed = time.time() - t
print('Time taken for Forward Backward =', elapsed)

print(obj[-1])
#plt.plot(obj)
#plt.imshow(x)

Time taken for Forward Backward = 196.80392360687256
0.9987908748831436


## Exact Test

In [ ]:
import matplotlib.pyplot as plt
M = 4                  
N = 4
m = 0.25*np.ones(M)
n = 0.25*np.ones(N)
c = np.array([[3.10112791, 2.01924761, 2.        , 2.01924761],
     [3.10112791, 3.        , 1.16637984, 1.        ],
     [3.        , 3.10112791, 1.16637984, 2.01924761],
     [1.        , 3.        , 2.33275967, 4.22075809]])
[x,obj] = generalised_forward_backward(c,m,n,10000)
plt.plot(obj)
print(obj[-1])
print(x)

## Entropic Regularisation for an Initial Guess

In [ ]:
def entropic_regularisation(m,n,c,iters_max,eps= 0.025):
    
    #fetch lengths of m and n.
    N = np.size(n)
    M = np.size(m)
    
    #Gibbs Kernel
    Geps = np.exp(-c/eps)
    
    cons = 1/np.sum(Geps)
    a = np.ones(M)*cons
    b = np.ones(N)*cons
    norm_difference = 100
    obj = []
    Kp = (1/(n + 1e-299)).reshape(-1,1) * Geps
    for i in range(iters_max):
        #anew = np.multiply(m,np.reciprocal(np.matmul(Geps, b)))
        #bnew = np.multiply(n,np.reciprocal(np.matmul(a.T, Geps).T))
        #anew = np.divide(m,np.matmul(Geps, b))
        #bnew = np.divide(n,np.matmul(a.T, Geps).T)
        bnew = np.divide(m, np.dot(Geps.T, a)+1e-299)
        anew = 1. / (np.dot(Kp,bnew)+1e-299)
                
        #anew = np.divide(m, Geps * b)
        #bnew = np.divide(n, (a.T * Geps).T)
        
        
        #x = a * Geps * b.T
        #u = anew * Geps * bnew.T
        x = np.multiply(a, np.multiply(Geps, b.T))
        u = np.multiply(anew, np.multiply(Geps, bnew.T))
        #x = a @ Geps @ b.T
        #u = anew @ Geps @ bnew.T
        
        norm_difference = np.linalg.norm(x-u)
        
        a = anew
        b = bnew
        
        obj.append((np.multiply(c,u)).sum())
    
    
    return x, obj

In [ ]:
M = 4                  
N = 4
m = 0.25*np.ones(M)
n = 0.25*np.ones(N)
c = np.array([[3.10112791, 2.01924761, 2.        , 2.01924761],
     [3.10112791, 3.        , 1.16637984, 1.        ],
     [3.        , 3.10112791, 1.16637984, 2.01924761],
     [1.        , 3.        , 2.33275967, 4.22075809]])
x, obj = entropic_regularisation(m,n,c,100000)
plt.plot(obj)
print(obj[-1])
print(x)

In [35]:
### from https://github.com/rflamary/POT/blob/master/ot/bregman.py ###
def sinkhorn_knopp(a, b, M, reg, numItermax=1000,
                   stopThr=1e-9, verbose=False, log=False, **kwargs):
    r"""
    Solve the entropic regularization optimal transport problem and return the OT matrix
    The function solves the following optimization problem:
    .. math::
        \gamma = arg\min_\gamma <\gamma,M>_F + reg\cdot\Omega(\gamma)
        s.t. \gamma 1 = a
             \gamma^T 1= b
             \gamma\geq 0
    where :
    - M is the (dim_a, dim_b) metric cost matrix
    - :math:`\Omega` is the entropic regularization term :math:`\Omega(\gamma)=\sum_{i,j} \gamma_{i,j}\log(\gamma_{i,j})`
    - a and b are source and target weights (histograms, both sum to 1)
    The algorithm used for solving the problem is the Sinkhorn-Knopp matrix scaling algorithm as proposed in [2]_
    Parameters
    ----------
    a : ndarray, shape (dim_a,)
        samples weights in the source domain
    b : ndarray, shape (dim_b,) or ndarray, shape (dim_b, n_hists)
        samples in the target domain, compute sinkhorn with multiple targets
        and fixed M if b is a matrix (return OT loss + dual variables in log)
    M : ndarray, shape (dim_a, dim_b)
        loss matrix
    reg : float
        Regularization term >0
    numItermax : int, optional
        Max number of iterations
    stopThr : float, optional
        Stop threshol on error (>0)
    verbose : bool, optional
        Print information along iterations
    log : bool, optional
        record log if True
    Returns
    -------
    gamma : ndarray, shape (dim_a, dim_b)
        Optimal transportation matrix for the given parameters
    log : dict
        log dictionary return only if log==True in parameters
    Examples
    --------
    >>> import ot
    >>> a=[.5, .5]
    >>> b=[.5, .5]
    >>> M=[[0., 1.], [1., 0.]]
    >>> ot.sinkhorn(a, b, M, 1)
    array([[0.36552929, 0.13447071],
           [0.13447071, 0.36552929]])
    References
    ----------
    .. [2] M. Cuturi, Sinkhorn Distances : Lightspeed Computation of Optimal Transport, Advances in Neural Information Processing Systems (NIPS) 26, 2013
    See Also
    --------
    ot.lp.emd : Unregularized OT
    ot.optim.cg : General regularized OT
    """

    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)
    M = np.asarray(M, dtype=np.float64)

    if len(a) == 0:
        a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
    if len(b) == 0:
        b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # init data
    dim_a = len(a)
    dim_b = len(b)

    if len(b.shape) > 1:
        n_hists = b.shape[1]
    else:
        n_hists = 0

    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if n_hists:
        u = np.ones((dim_a, n_hists)) / dim_a
        v = np.ones((dim_b, n_hists)) / dim_b
    else:
        u = np.ones(dim_a) / dim_a
        v = np.ones(dim_b) / dim_b

    # print(reg)

    # Next 3 lines equivalent to K= np.exp(-M/reg), but faster to compute
    K = np.empty(M.shape, dtype=M.dtype)
    np.divide(M, -reg, out=K)
    np.exp(K, out=K)

    # print(np.min(K))
    tmp2 = np.empty(b.shape, dtype=M.dtype)

    Kp = (1 / (a+ 1e-299)).reshape(-1, 1) * K
    cpt = 0
    err = 1
    while (err > stopThr and cpt < numItermax):
        uprev = u
        vprev = v

        KtransposeU = np.dot(K.T, u)
        v = np.divide(b, (KtransposeU+ 1e-299))
        u = 1. / (np.dot(Kp, v)+ 1e-299)

        if (np.any(KtransposeU == 0)
                or np.any(np.isnan(u)) or np.any(np.isnan(v))
                or np.any(np.isinf(u)) or np.any(np.isinf(v))):
            # we have reached the machine precision
            # come back to previous solution and quit loop
            print('Warning: numerical errors at iteration', cpt)
            u = uprev
            v = vprev
            break
        if cpt % 10 == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            if n_hists:
                np.einsum('ik,ij,jk->jk', u, K, v, out=tmp2)
            else:
                # compute right marginal tmp2= (diag(u)Kdiag(v))^T1
                np.einsum('i,ij,j->j', u, K, v, out=tmp2)
            err = np.linalg.norm(tmp2 - b)  # violation of marginal
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % 200 == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))
        cpt = cpt + 1
    if log:
        log['u'] = u
        log['v'] = v

    if n_hists:  # return only loss
        res = np.einsum('ik,ij,jk,ij->k', u, K, v, M)
        if log:
            return res, cpt, log
        else:
            return res, cpt

    else:  # return OT matrix

        if log:
            return u.reshape((-1, 1)) * K * v.reshape((1, -1)), cpt, log
        else:
            return u.reshape((-1, 1)) * K * v.reshape((1, -1)), cpt, (np.multiply(M,u.reshape((-1, 1)) * K * v.reshape((1, -1)))).sum()


In [ ]:
M = 4                  
N = 4
m = 0.25*np.ones(M)
n = 0.25*np.ones(N)
c = np.array([[3.10112791, 2.01924761, 2.        , 2.01924761],
     [3.10112791, 3.        , 1.16637984, 1.        ],
     [3.        , 3.10112791, 1.16637984, 2.01924761],
     [1.        , 3.        , 2.33275967, 4.22075809]])
%timeit -r 20 -n 20 sinkhorn_knopp(m,n,c,0.005)

In [36]:
x = np.linspace(0,1,1000)
y = np.linspace(1,2,1000)
len_x = np.size(x)
len_y = np.size(y)

c = np.zeros((len_x,len_y))
for i in range(len_x):
    for j in range(len_y):
        c[i,j] = (y[j]-x[i])**2

m = np.ones(len_x)/len_x
n = np.ones(len_y)/len_y
M = len_x
N = len_y
%timeit -r 1 -n 1 sinkhorn_knopp(m,n,c,0.005)
sinkhorn_knopp(m,n,c,0.005)

78 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


(array([[2.14034157e-05, 2.11001659e-05, 2.07982920e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.11001660e-05, 2.08095516e-05, 2.05200582e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.07982920e-05, 2.05200582e-05, 2.02427037e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [6.41085604e-92, 9.42458775e-92, 1.38531238e-91, ...,
         2.02427029e-05, 2.05200574e-05, 2.07982911e-05],
        [4.35795298e-92, 6.40918702e-92, 9.42458775e-92, ...,
         2.05200574e-05, 2.08095508e-05, 2.11001651e-05],
        [2.96202057e-92, 4.35795298e-92, 6.41085604e-92, ...,
         2.07982912e-05, 2.11001651e-05, 2.14034148e-05]]),
 721,
 1.0023757748875959)

In [37]:
%timeit -r 1 -n 1 sinkhorn_knopp(m,n,c,0.005)

1.59 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [38]:
%timeit -r 1 -n 1 generalised_forward_backward(c,m,n,1000)

4min 11s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [39]:
%timeit -r 1 -n 1 primal_dual(c,m,n,1000)

6min 53s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
